In [1]:
pip install requests transformers torch tqdm newspaper3k


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install lxml[html_clean]


In [4]:
pip install newspaper3k

##First Attempts

In [5]:
# summarize_gnews.py

import requests
from datetime import datetime, timedelta
from transformers import pipeline
from newspaper import Article
import json
from tqdm import tqdm

GNEWS_API_KEY = "API KEY"
CATEGORIES = ["world", "business", "technology"]
MAX_RESULTS = 30  # total per day across all categories

summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small", framework="pt")

def fetch_articles(category):
    date_yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://gnews.io/api/v4/search?q={category}&lang=en&from={date_yesterday}&max=10&token={GNEWS_API_KEY}"
    res = requests.get(url)
    return res.json().get("articles", [])

def extract_full_text(article_url):
    try:
        a = Article(article_url)
        a.download()
        a.parse()
        return a.text
    except:
        return None

def summarize_text(text):
    if len(text.split()) < 40:
        return text.strip()  # too short to summarize
    trimmed = " ".join(text.split()[:300])  # reduce token load
    summary = summarizer("summarize: " + trimmed, max_length=50, min_length=20, do_sample=False)
    return summary[0]["summary_text"].strip()

def main():
    summarized_articles = []

    for category in CATEGORIES:
        articles = fetch_articles(category)
        for article in tqdm(articles, desc=f"Processing {category}"):
            full_text = extract_full_text(article["url"]) or article.get("description", "")
            if not full_text:
                continue
            summary = summarize_text(full_text)
            summarized_articles.append({
                "title": article["title"],
                "summary": summary,
                "url": article["url"],
                "category": category,
                "published": article.get("publishedAt", "")
            })
            if len(summarized_articles) >= MAX_RESULTS:
                break

    with open("daily_summary.json", "w", encoding="utf-8") as f:
        json.dump(summarized_articles, f, indent=2)

    print(f"✅ Saved {len(summarized_articles)} summaries to daily_summary.json")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
Processing technology:  90%|█████████ | 9/10 [00:41<00:04,  4.64s/it]

✅ Saved 30 summaries to daily_summary.json


In [6]:
# summarize_gnews.py

import requests
from datetime import datetime, timedelta
from transformers import pipeline
from newspaper import Article
import json
from tqdm import tqdm

# === CONFIG ===
GNEWS_API_KEY = "API KEY"
CATEGORIES = ["world", "business", "technology"]
MAX_RESULTS = 30  # total across all categories
ARTICLES_PER_CATEGORY = 10

# === SUMMARIZATION MODEL ===
summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small", framework="pt")

# === FUNCTIONS ===

def fetch_articles(category):
    """Fetches articles from GNews API."""
    date_yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://gnews.io/api/v4/search?q={category}&lang=en&from={date_yesterday}&max={ARTICLES_PER_CATEGORY}&token={GNEWS_API_KEY}"
    res = requests.get(url)
    if res.status_code != 200:
        return []
    return res.json().get("articles", [])

def extract_full_text(article_url):
    """Attempts to extract full article text using newspaper3k."""
    try:
        a = Article(article_url)
        a.download()
        a.parse()
        return a.text
    except:
        return None

def summarize_text(text):
    """Summarizes a block of text using T5-small."""
    if len(text.split()) < 40:
        return text.strip()
    trimmed = " ".join(text.split()[:300])
    summary = summarizer("summarize: " + trimmed, max_length=50, min_length=20, do_sample=False)
    return summary[0]["summary_text"].strip()

# === MAIN SCRIPT ===

def main():
    summarized_articles = []

    for category in CATEGORIES:
        articles = fetch_articles(category)
        for article in tqdm(articles, desc=f"Processing {category}"):
            full_text = extract_full_text(article["url"]) or article.get("description", "")
            if not full_text:
                continue
            summary = summarize_text(full_text)
            summarized_articles.append({
                "title": article["title"],
                "summary": summary,
                "url": article["url"],
                "category": category,
                "published": article.get("publishedAt", "")
            })
            if len(summarized_articles) >= MAX_RESULTS:
                break
        if len(summarized_articles) >= MAX_RESULTS:
            break

    with open("daily_summary.json", "w", encoding="utf-8") as f:
        json.dump(summarized_articles, f, indent=2)

    print(f"Saved {len(summarized_articles)} summaries to daily_summary.json")

if __name__ == "__main__":
    main()


Device set to use cpu
Processing technology:  90%|█████████ | 9/10 [00:43<00:04,  4.88s/it]

Saved 30 summaries to daily_summary.json


In [17]:
import requests
from datetime import datetime, timedelta
from transformers import pipeline
from newspaper import Article
import json
from tqdm import tqdm

# === CONFIG ===
GNEWS_API_KEY = "API KEY"
CATEGORIES = ["world", "business", "technology"]
ARTICLES_PER_CATEGORY = 30

# === SUMMARIZATION MODEL ===
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn", framework="pt")

# === FUNCTIONS ===

def fetch_articles(category):
    """Fetches articles from GNews API."""
    date_yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://gnews.io/api/v4/search?q={category}&lang=en&from={date_yesterday}&max={ARTICLES_PER_CATEGORY}&token={GNEWS_API_KEY}"
    res = requests.get(url)
    if res.status_code != 200:
        return []
    return res.json().get("articles", [])

def extract_full_text(article_url):
    """Attempts to extract full article text using newspaper3k."""
    try:
        a = Article(article_url)
        a.download()
        a.parse()
        return a.text
    except:
        return None

def summarize_paragraph(text):
    """Summarizes a long block of concatenated article texts."""
    if len(text.split()) < 40:
        return text.strip()
    trimmed = " ".join(text.split()[:500])  # Cap to avoid overloading
    result = summarizer(trimmed, max_new_tokens=120, do_sample=False)
    return result[0]["summary_text"].strip()

# === MAIN SCRIPT ===

def main():
    category_summaries = {}

    for category in CATEGORIES:
        all_text = []
        articles = fetch_articles(category)

        for article in tqdm(articles, desc=f"Processing {category}"):
            full_text = extract_full_text(article["url"]) or article.get("description", "")
            if full_text:
                all_text.append(full_text.strip())

        combined_text = ". ".join(all_text)
        if combined_text:
            category_summary = summarize_paragraph(combined_text)
            category_summaries[category] = category_summary

    with open("daily_summary.json", "w", encoding="utf-8") as f:
        json.dump(category_summaries, f, indent=2)

    print(f"Saved category-wise paragraph summaries to daily_summary.json")

if __name__ == "__main__":
    main()


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Processing technology: 100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


Saved category-wise paragraph summaries to daily_summary.json


In [19]:
import requests
from datetime import datetime, timedelta
from transformers import pipeline
from newspaper import Article
import json
from tqdm import tqdm
import re

# === CONFIG ===
GNEWS_API_KEY = "API KEY"
CATEGORY_QUERIES = {
    "world": "global OR international OR diplomacy",
    "business": "economy OR finance OR market OR stock",
    "technology": "AI OR tech OR startup OR software"
}
ARTICLES_PER_CATEGORY = 30
MIN_WORDS_REQUIRED = 80

# === SUMMARIZATION MODEL ===
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn",
    framework="pt"
)

def fetch_articles(query):
    """Fetches articles from GNews API."""
    date_yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://gnews.io/api/v4/search?q={query}&lang=en&from={date_yesterday}&max={ARTICLES_PER_CATEGORY}&token={GNEWS_API_KEY}"
    res = requests.get(url)
    if res.status_code != 200:
        return []
    return res.json().get("articles", [])

def extract_full_text(article_url):
    """Extracts full article text using newspaper3k."""
    try:
        a = Article(article_url)
        a.download()
        a.parse()
        return a.text
    except:
        return None

def clean_text(text):
    """Removes weird characters, excessive whitespace, etc."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # remove non-ASCII
    return text.strip()

def summarize_paragraph(text, max_tokens=1024):
    """Chunks and summarizes large text if needed."""
    words = text.split()
    chunks = [' '.join(words[i:i+500]) for i in range(0, len(words), 500)]
    summaries = []
    for chunk in chunks:
        try:
            summary = summarizer(chunk, max_new_tokens=160, do_sample=False)
            summaries.append(summary[0]["summary_text"].strip())
        except Exception:
            continue
    return ' '.join(summaries)

def main():
    category_summaries = {}

    for category, query in CATEGORY_QUERIES.items():
        print(f"\n=== Processing {category.upper()} ===")
        all_text = []
        articles = fetch_articles(query)

        for article in tqdm(articles, desc=f"{category.title()} Articles"):
            full_text = extract_full_text(article["url"]) or article.get("description", "")
            if not full_text or len(full_text.split()) < MIN_WORDS_REQUIRED:
                continue
            clean = clean_text(full_text)
            if clean:
                all_text.append(clean)

        combined_text = ". ".join(all_text)
        if combined_text:
            category_summary = summarize_paragraph(combined_text)
            category_summaries[category] = category_summary
        else:
            category_summaries[category] = "No substantial content available to summarize."

    with open("daily_summary.json", "w", encoding="utf-8") as f:
        json.dump(category_summaries, f, indent=2)

    print(f"\n✅ Saved category-wise paragraph summaries to daily_summary.json")

if __name__ == "__main__":
    main()


Device set to use cpu



=== Processing WORLD ===


World Articles: 100%|██████████| 10/10 [00:06<00:00,  1.44it/s]



=== Processing BUSINESS ===


Business Articles: 100%|██████████| 10/10 [00:13<00:00,  1.31s/it]



=== Processing TECHNOLOGY ===


Technology Articles: 100%|██████████| 10/10 [00:08<00:00,  1.20it/s]



✅ Saved category-wise paragraph summaries to daily_summary.json


In [21]:
import requests
from datetime import datetime, timedelta
from transformers import pipeline
from newspaper import Article
from tqdm import tqdm
import json
import re

# === CONFIG ===
GNEWS_API_KEY = "API KEY"
CATEGORY_QUERIES = {
    "world": "global OR international OR diplomacy",
    "business": "economy OR finance OR market OR stock",
    "technology": "AI OR tech OR startup OR software"
}
ARTICLES_PER_CATEGORY = 25
MIN_WORDS_REQUIRED = 100
MAX_SUMMARIES_PER_CATEGORY = 5

# === SUMMARIZATION MODEL ===
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn",
    framework="pt"
)

def fetch_articles(query):
    """Fetch articles from GNews API."""
    date_yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://gnews.io/api/v4/search?q={query}&lang=en&from={date_yesterday}&max={ARTICLES_PER_CATEGORY}&token={GNEWS_API_KEY}"
    res = requests.get(url)
    return res.json().get("articles", []) if res.status_code == 200 else []

def extract_full_text(url):
    """Extract full text from article using newspaper3k."""
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        return None

def clean_text(text):
    """Remove non-ASCII and excessive whitespace."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.strip()

def is_junk(text):
    """Filter out articles with only tickers, locations, or lists."""
    return (
        len(text.split()) < MIN_WORDS_REQUIRED or
        re.search(r'\b(Dow|TSX|Nikkei|NASDAQ|USD|S&P)\b', text) or
        re.search(r'\b[A-Z][a-z]+, [A-Z][a-z]+, [A-Z][a-z]+\b', text)  # country dump pattern
    )

def summarize(text):
    """Summarize text with the transformer model."""
    try:
        return summarizer(text, max_new_tokens=120, do_sample=False)[0]["summary_text"].strip()
    except:
        return None

def process_category(name, query):
    """Process a single category and return a concise summary."""
    print(f"\n=== Processing {name.upper()} ===")
    articles = fetch_articles(query)
    summaries = []

    for article in tqdm(articles, desc=f"{name.title()} Articles"):
        raw_text = extract_full_text(article["url"]) or article.get("description", "")
        if not raw_text:
            continue

        cleaned = clean_text(raw_text)
        if is_junk(cleaned):
            continue

        summary = summarize(cleaned[:1024])  # truncate to fit model input
        if summary:
            summaries.append(summary)

        if len(summaries) >= MAX_SUMMARIES_PER_CATEGORY:
            break

    if summaries:
        return ' '.join(summaries)
    return "No substantial content available to summarize."

def main():
    result = {}
    for category, query in CATEGORY_QUERIES.items():
        result[category] = process_category(category, query)

    with open("daily_summary.json", "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2)
    print("\n✅ Saved daily_summary.json")

if __name__ == "__main__":
    main()


Device set to use cpu



=== Processing WORLD ===


World Articles:  60%|██████    | 6/10 [01:55<01:16, 19.24s/it]



=== Processing BUSINESS ===


Business Articles: 100%|██████████| 10/10 [01:09<00:00,  6.93s/it]



=== Processing TECHNOLOGY ===


Technology Articles:  40%|████      | 4/10 [01:37<02:26, 24.36s/it]


✅ Saved daily_summary.json


##Final

In [22]:
import requests
from datetime import datetime, timedelta
from newspaper import Article
from transformers import pipeline
import json
import re
from tqdm import tqdm

# === CONFIG ===
GNEWS_API_KEY = "API KEY"
CATEGORY_QUERIES = {
    "world": "global OR international OR diplomacy",
    "business": "economy OR finance OR market OR stock",
    "technology": "AI OR tech OR startup OR software"
}
ARTICLES_PER_CATEGORY = 30
MIN_WORDS_REQUIRED = 80
MAX_SUMMARIES = 5

# === SUMMARIZER ===
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn",
    framework="pt"
)

def fetch_articles(query):
    date_yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://gnews.io/api/v4/search?q={query}&lang=en&from={date_yesterday}&max={ARTICLES_PER_CATEGORY}&token={GNEWS_API_KEY}"
    res = requests.get(url)
    if res.status_code != 200:
        return []
    return res.json().get("articles", [])

def extract_full_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        return None

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # remove non-ASCII
    text = re.sub(r"(The United States.*?islands\.)", "", text, flags=re.IGNORECASE)
    text = re.sub(r"(The U\.S\. Virgin Islands.*?\.)", "", text, flags=re.IGNORECASE)
    return text.strip()

def is_valid_text(text):
    return text and len(text.split()) >= MIN_WORDS_REQUIRED

def is_repetitive(text):
    return (
        text.count("United States") > 2 or
        text.lower().count("virgin islands") > 2 or
        len(set(text.split('. '))) < len(text.split('. ')) * 0.6
    )

def summarize_text(text):
    try:
        summary = summarizer(text[:1024], max_new_tokens=120, do_sample=False)[0]['summary_text'].strip()
        return summary
    except:
        return None

def process_category(name, query):
    print(f"\n=== {name.upper()} ===")
    summaries = []
    articles = fetch_articles(query)

    for article in tqdm(articles, desc=f"{name.title()} Articles"):
        full_text = extract_full_text(article["url"]) or article.get("description", "")
        cleaned = clean_text(full_text)

        if is_valid_text(cleaned):
            summary = summarize_text(cleaned)
            if summary and not is_repetitive(summary):
                summaries.append(summary)

        if len(summaries) >= MAX_SUMMARIES:
            break

    return summaries if summaries else ["No reliable summaries found."]

def main():
    results = {}

    for category, query in CATEGORY_QUERIES.items():
        results[category] = process_category(category, query)

    with open("daily_summary.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    print("\n✅ Summaries saved to daily_summary.json")

if __name__ == "__main__":
    main()


Device set to use cpu



=== WORLD ===


World Articles:  50%|█████     | 5/10 [02:16<02:16, 27.24s/it]



=== BUSINESS ===


Business Articles:  60%|██████    | 6/10 [02:05<01:23, 20.83s/it]



=== TECHNOLOGY ===


Technology Articles:  40%|████      | 4/10 [01:37<02:26, 24.36s/it]


✅ Summaries saved to daily_summary.json
